# Basic analysis of *"GIS data"*

Place data in `file_parser/data` directory.

What is needed for a model:
* Junctions
    * **Elevation** - The elevation in feet (meters) above some common reference of the junction. This is a **required** property. Elevation is used only to compute pressure at the junction. It does not affect any other computed quantity.
    // * **Base Demand** -The average or nominal demand for water by the main category of consumer at the junction, as measured in the current flow units. A negative value is used to indicate an external source of flow into the junction. If left blank then demand is assumed to be zero.
* Pipes
    *  **Length** - The actual length of the pipe in feet (meters). This is a **required** property.
    * **Diam** - The pipe diameter in inches (mm). This is a **required** property.
    * **Roughness** - The roughness coefficient of the pipe. It is unitless for Hazen-Williams or Chezy-Manning roughness and has units of millifeet (mm) for Darcy-Weisbach roughness. This is a **required** property.


In [1]:
import pandas as pd
import geopandas as gpd

# attributestablebrowsercontroller (1)

In [7]:
controller = pd.read_excel(
    "./data/attributestablebrowsercontroller_1.xlsx", index_col=0
)

C:\Users\jolka\OneDrive\Pulpit\studia\sem_8\MISS\System-modelling-and-simulation\venv\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [ ]:
controller.head()

In [9]:
controller.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1033 entries, 68085 to 95556
Data columns (total 35 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Stan eksploatacji        1033 non-null   object        
 1   Funkcja                  1030 non-null   object        
 2   Średnica nominalna [mm]  771 non-null    float64       
 3   Materiał                 41 non-null     object        
 4   Rok budowy               0 non-null      float64       
 5   Rzędna początku          0 non-null      float64       
 6   Rzędna końca             0 non-null      float64       
 7   Miasto                   602 non-null    object        
 8   Ulica                    602 non-null    object        
 9   Numer domu               204 non-null    object        
 10  Przebieg orientacyjny    0 non-null      float64       
 11  Długość pomierzona [m]   0 non-null      float64       
 12  Własność                 1033 non-

Drop not important columns

In [20]:
columns_to_drop = [
    "Rok budowy",
    "Rzędna początku",
    "Rzędna końca",
    "Miasto",
    "Ulica",
    "Przebieg orientacyjny",
    "Długość pomierzona [m]",
    "Własność",
    "Źródło danych",
    "Inspektor",
    "Kierownik budowy",
    "Inwestor",
    "Wykonawca",
    "Numer uzg. proj.",
    "Data uzg. proj.",
    "Miasto (rozwinięte)",
    "Numer domu (rozwinięte)",
    "Gmina",
    "Nr środka trwałego",
    "Użytkownik modyfikujący",
    "Użytkownik tworzący",
    "Data utworzenia",
    "numer",
    "automat",
]

controller = controller.drop(columns=columns_to_drop)

In [22]:
controller.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1014 entries, 68085 to 95556
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Stan eksploatacji        1014 non-null   object        
 1   Funkcja                  1014 non-null   object        
 2   Średnica nominalna [mm]  763 non-null    float64       
 3   Materiał                 41 non-null     object        
 4   Numer domu               203 non-null    object        
 5   Uwagi                    41 non-null     object        
 6   Długość geometrii        1014 non-null   float64       
 7   Ulica (rozwinięte)       596 non-null    object        
 8   Opis lokalizacji         1014 non-null   object        
 9   Nr działki               999 non-null    object        
 10  Data modyfikacji         692 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(2), object(8)
memory usage: 95.1+ KB


In [ ]:
controller.head()

In [11]:
controller["Stan eksploatacji"].value_counts()

Stan eksploatacji
Czynny          1014
Nieczynny         10
Projektowany       9
Name: count, dtype: int64

In [12]:
controller = controller[controller["Stan eksploatacji"] == "Czynny"]

In [13]:
controller["Stan eksploatacji"].value_counts()

Stan eksploatacji
Czynny    1014
Name: count, dtype: int64

In [24]:
controller["Funkcja"].value_counts()

Funkcja
Przyłącze               494
Rozdzielczy             281
Inny                    165
Przyłącze hydrantowe     71
Magistralny               3
Name: count, dtype: int64

In [25]:
controller["Średnica nominalna [mm]"].value_counts()  # Diam

Średnica nominalna [mm]
32.0     241
40.0     207
100.0     65
90.0      64
110.0     59
80.0      35
50.0      27
63.0      19
150.0     17
25.0      13
160.0     11
20.0       4
75.0       1
Name: count, dtype: int64

In [26]:
controller["Materiał"].value_counts()

Materiał
PVC            19
Brak danych    13
PE              9
Name: count, dtype: int64

In [27]:
controller["Uwagi"].value_counts()  # duplicates

Uwagi
PCV                                                        19
PE                                                          9
zmiana średnicy na podstawie map branżowych z 90 na 110     7
zmiana średnicy na podstawie map branżowych z 110 na 90     2
brak                                                        1
data pomiaru: 28.02.1997                                    1
347/03                                                      1
209/08                                                      1
Name: count, dtype: int64

In [28]:
controller["Długość geometrii"].value_counts()  # what is this?

Długość geometrii
7.67      3
5.88      3
2.17      3
11.16     3
0.36      3
         ..
0.88      1
129.18    1
137.76    1
114.38    1
42.89     1
Name: count, Length: 943, dtype: int64

In [29]:
controller["Opis lokalizacji"].value_counts()  # ...

Opis lokalizacji
podziemny    1014
Name: count, dtype: int64

# attributestablebrowsercontroller

In [33]:
controller = pd.read_excel("./data/attributestablebrowsercontroller.xlsx")

C:\Users\jolka\OneDrive\Pulpit\studia\sem_8\MISS\System-modelling-and-simulation\venv\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [34]:
controller.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 797 entries, 0 to 796
Data columns (total 28 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   address_point_id      797 non-null    object        
 1   id_address_point_map  797 non-null    int64         
 2   id_msc_zamontu        797 non-null    int64         
 3   wodomierz             797 non-null    bool          
 4   id_punktu             797 non-null    int64         
 5   id_zamontu            797 non-null    int64         
 6   id_wodomierza         797 non-null    int64         
 7   adres                 797 non-null    object        
 8   Średnica              786 non-null    float64       
 9   Producent             786 non-null    object        
 10  Nr fabryczny          797 non-null    object        
 11  Główny                143 non-null    float64       
 12  srednia_dobowa        712 non-null    float64       
 13  Data zamontu        

In [35]:
controller.head()

,address_point_id,id_address_point_map,id_msc_zamontu,wodomierz,id_punktu,id_zamontu,id_wodomierza,adres,Średnica,Producent,...,tel,Numer umowy,Data zawarcia,Data wygaśnięcia,Typ umowy,Id wew.,nr_odbiorcy,email,nr_dzialki,aktywny
0,24,16481,54735,True,223474,28326,0,"Jasień, Zielna, 24",NaN,NaN,...,NaN,123474,2019-01-07,4000-02-01 00:00:00,NaN,19588,223474,NaN,1407,False
1,115,16390,54137,True,222934,27606,27726,"Jasień, Wiedeńska, 115",20.0,SENSUS,...,NaN,122934,2020-09-28,4000-02-01 00:00:00,NaN,19589,222934,NaN,1826/1,True
2,115,16390,54137,True,222934,27605,16793,"Jasień, Wiedeńska, 115",20.0,"POWOGAZ, METRON",...,NaN,122934,2019-01-11,2020-09-28 00:00:00,NaN,19589,222934,NaN,1826/1,False
3,47,16149,63305,True,219811,23393,25139,"Jasień, Ks. Mazurkiewicza, 47",20.0,SENSUS,...,604254690,119811,2019-09-03,4000-02-01 00:00:00,NaN,19740,219811,NaN,1127/6,True
4,47,16149,63305,True,219811,23392,3332,"Jasień, Ks. Mazurkiewicza, 47",20.0,"POWOGAZ, METRON",...,604254690,119811,2019-01-09,2019-09-03 00:00:00,NaN,19740,219811,NaN,1127/6,False


In [36]:
controller["wodomierz"].value_counts()

wodomierz
True    797
Name: count, dtype: int64

Dane do kalibracji.
Problem: brak adresu w przypadku popszedniej tabeli. Możliwy analogiczny problem w przypadku innych plików.